<a href="https://colab.research.google.com/github/ekrombouts/GenCareAI/blob/main/notebooks/100_note_generation/140_GenerateClientRecords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GenCare AI: Generating client records

**Author:** Eva Rombouts  
**Date:** 2024-06-15  
**Updated:** 2024-09-01  
**Version:** 1.2

### Description
This script generates synthetic care notes and summaries for clients in a psychogeriatric ward using the OpenAI GPT-3.5-turbo model.  
The care notes are based on client profiles and scenarios generated in earlier scripts in this repo. The goal is to add as much story and variation to the notes as possible, without letting the model produce outputs that are overly creative.  
To achieve this, I use structured prompts (to help the model understand the expected content and its creative liberties), example libraries and memory integration.  
The output parser uses Pydantic models to structure and validate the care notes, ensuring proper format and content.  
Chroma is used to retrieve example care notes that are representative of the client profile and scenario. 

The script processes client profiles and scenarios, generates care notes, and updates summaries accordingly, creating comprehensive client records.  
The goal is to create a diverse and realistic dataset for NLP experiments in nursing homes.

**Please note** that generating data with OpenAI is not free. Generating records for 24 clients with a mean of 8 months, 5 iterations per month takes about 3 hours en costs appr $2,- with gpt-3.5

## Setup

In [2]:
!pip install GenCareAI
from GenCareAI.GenCareAIUtils import GenCareAISetup

setup = GenCareAISetup()

if setup.environment == 'Colab':
        !pip install -q langchain langchain-openai langchain-community langchain-chroma

In [3]:
# Imports
import random
import pandas as pd
from pprint import pprint
from typing import List

from langchain.output_parsers import PydanticOutputParser, CommaSeparatedListOutputParser
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_community.callbacks import get_openai_callback

In [4]:
# Paths to various data files and constants for the model and temperature settings.
path_db_gcai = setup.get_file_path('data/chroma_db_gcai_notes')
ward_name = 'Apollo' # Make sure to use the same ward-name for which clientprofiles and -scenarios have been generated
path_profiles = setup.get_file_path(f'data/gcai_client_profiles_{ward_name}.csv')
path_scenarios = setup.get_file_path(f'data/gcai_client_subscenarios_{ward_name}.csv')
path_notes = setup.get_file_path(f'data/gcai_client_notes_{ward_name}.csv')
path_summaries = setup.get_file_path(f'data/gcai_client_summaries_{ward_name}.csv')

collection_name = 'anonymous_notes'
model = 'gpt-3.5-turbo-0125'
model_embeddings = 'text-embedding-ada-002'
temp = 1.1

verbose = True # Set to True for debugging / printing
sample_client = 2
sep_line = 100*'-'

### Load data

In the notebooks [110_GenerateClientProfiles.ipynb]() and [120_GenerateClientScenarios.ipynb](), datasets are generated with client profiles and scenarios, respectively.

- ***df_profiles***: Contains one row per client. The row describes the type of dementia the client is diagnosed with, the ADL care needs, medical symptoms and diseases, mobility, and behavior.
  
- ***df_scenarios***: Each client has zero or more scenario lines. These are referred to as month numbers, but they do not necessarily correspond to actual months.

In [5]:
# Load scenarios and profiles from CSV files
df_profiles = pd.read_csv(path_profiles, encoding='utf-8')
df_scenarios = pd.read_csv(path_scenarios, encoding='utf-8')

if verbose:
    print(df_profiles.info())
    print(df_scenarios.info())
    test_client = df_profiles.iloc[sample_client] # We will be using this test client throughout the script

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   client_id      24 non-null     int64 
 1   naam           24 non-null     object
 2   type_dementie  24 non-null     object
 3   somatiek       24 non-null     object
 4   adl            24 non-null     object
 5   mobiliteit     24 non-null     object
 6   gedrag         24 non-null     object
dtypes: int64(1), object(6)
memory usage: 1.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   client_id           449 non-null    int64 
 1   period              449 non-null    int64 
 2   sub_period          449 non-null    int64 
 3   events_description  449 non-null    object
dtypes: int64(3), object(1)
memory usage: 14.2+ KB
None


### Functions to display the data

In [6]:
# Function to format the client’s profile information as a single string
def profile_as_string(profile_row, display_name=True):
    profile = ""
    if display_name:
        profile += f"Naam: {profile_row['naam']}\n"
    profile += (f"Type Dementie: {profile_row['type_dementie']}\n"
                f"Lichamelijke klachten: {profile_row['somatiek']}\n"
                f"ADL: {profile_row['adl']}\n"
                f"Mobiliteit: {profile_row['mobiliteit']}\n"
                f"Cognitie/gedrag: {profile_row['gedrag']}")
    return profile

if verbose:
    print(profile_as_string(profile_row=test_client, display_name=True))

Naam: Meneer Emiel Bos
Type Dementie: Vasculaire dementie
Lichamelijke klachten: Chronisch hartfalen
ADL: Beperkte hulp bij wassen en aankleden, eet zelfstandig
Mobiliteit: Gebruikt een wandelstok, langzaam lopend
Cognitie/gedrag: Meneer Bos is vaak gefrustreerd en kan soms verbaal agressief zijn, met perioden van diepe concentratie waarbij hij niet reageert.


In [7]:
# Function to display the scenario information for a given client and month
def scenario_as_string(profile_row, period=1, sub_period=1):
    client_id = profile_row['client_id']
    return df_scenarios.loc[
        (df_scenarios['client_id'] == client_id) &
        (df_scenarios['period'] == period) & 
        (df_scenarios['sub_period'] == sub_period), 
        'events_description'
    ].iloc[0]

if verbose:
    # print(scenario_as_string(profile_row=test_client))
    print(scenario_as_string(profile_row=test_client, period=2, sub_period=1))

Gedurende deze periode vertoont meneer Bos tekenen van gewichtsverlies door zijn beperkte mobiliteit en lichamelijke klachten. Er is gestart met fysiotherapie om zijn kracht en mobiliteit te verbeteren.


### Define Pydantic models

For parsing, we use pydantic models. These are remarkably well understood by the LLM, and offer several benefits:
- It enforces consistent output structure
- It automates parsing of the output
- The field description further guides the LLM model in generating appropriate responses. 
- It catches invalid data early, preventing downstream issues.

In our main prompt defined below, we ask the LLM to generate multiple care notes. Our structure consists of two classes: one to structure a single note (CareNote) and a second class defined as a list of care notes (CareNotes).

CareNote represents a single care note with the fields dag, tijd, and rapportage:
- **'dag'**: Even though the sequence number of the day ("dag") isn't meaningful, it forces the model to respond with the number of days requested.  
- **'tijd'**: The field 'time' ("tijd") was chosen over 'daypart' ("dagdeel") because the model tended to link 'daypart' to breakfast, lunch, and dinner, which led to many notes describing lunch details.  
- **'rapportage'**: This is the main challenge, what it's all about... 

CareNotes is a container for multiple care notes, consisting of a single field **'notes'**, which is a list of CareNote instances.

In [8]:
#Structure for a single care note
class CareNote(BaseModel):
    dag: int = Field(description="volgnummer dag")
    tijd: str = Field(description="tijd van de rapportage (hh:mm)")
    rapportage: str = Field(description="Inhoud van de rapportage. Een rapportage beschrijft over het algemeen één zorgaspect, soms meer")

# Structure for multiple notes
class CareNotes(BaseModel):
    notes: List[CareNote]

### Model initialization

The temperature and model settings can be configured in the constants section. I selected the OpenAI GPT-3.5-turbo model for cost-efficiency. The temperature setting of 1.1 was determined through trial and error.

In [9]:
# Initialize OpenAI Chat model
model = ChatOpenAI(api_key=setup.get_openai_key(), temperature=temp, model=model)

In [10]:
# Initialize CSV and pydantic parsers
csv_parser = CommaSeparatedListOutputParser()
csv_format_instructions = csv_parser.get_format_instructions()
pyd_parser = PydanticOutputParser(pydantic_object=CareNotes)

## Setting up the example library

### Setup Chroma

To improve the contextual relevance and diversity of the generated care notes, we will dynamically inject example notes into the prompts. 

Chroma is used in this context to select example care notes that are representative of the client profile and scenario. By using a vector database, the system can efficiently find example notes that match the specific characteristics of each client. The Chroma vector database, created [here](), contains a variety of (synthetic) notes for anonymous clients. The retriever queries this database, allowing the processing functions to access and integrate the stored example notes.  
By adding these dynamically selected examples to the prompt, this approach aims to make the generated care notes more contextually appropriate and diverse.

Our approach is:
- Initialize the Chroma Vector Database.
- Generate Keywords: Using the LLM, keywords are generated from a client’s profile and scenario. These keywords are used to query the Chroma vector database for relevant example notes.
- Retrieve Example Notes: Utilizing a retriever to search the vector database with the generated keywords, supplemented by additional neutral terms to ensure a broad coverage of relevant care notes.
- Filter by Gender: Filter the retrieved example notes to exclude those with gender-specific pronouns that do not match the client’s gender, ensuring the relevance of the examples.
- Sample Examples: Randomly sample a subset of the filtered example notes to inject into the prompts, enhancing the diversity of the generated care notes.

In [11]:
# Initialize Chroma vector database
vectordb = Chroma(persist_directory=path_db_gcai,
                  embedding_function=OpenAIEmbeddings(api_key=setup.get_openai_key(), model=model_embeddings),
                  collection_name = collection_name
                  )

In [12]:
# Set up a retriever for document querying
retriever = vectordb.as_retriever(search_kwargs={"k": 20})

### Keywords prompt & chain

In [29]:
# template to generate key words from a client’s profile and scenario for retrieving example notes
PT_keywords = PromptTemplate(
    template = """
Geef vijf korte zinnen die de kern weergeven van onderstaand scenario. 
Noem geen naam

Scenario:
{scenario}

{format_instructions}
""",
    input_variables=["scenario"],
    partial_variables={"format_instructions": csv_format_instructions},)

# Format the prompt for the example library
if verbose:
    P_keywords = PT_keywords.format(
        scenario=scenario_as_string(test_client, period=1, sub_period=1))
    print(P_keywords)


Geef vijf korte zinnen die de kern weergeven van onderstaand scenario. 
Noem geen naam

Scenario:
Meneer Bos wordt opgenomen in het verpleeghuis vanwege zijn vasculaire dementie en chronisch hartfalen. Hij krijgt begeleiding bij het wassen en aankleden, maar kan nog zelfstandig eten. Zijn mobiliteit is beperkt en hij maakt gebruik van een wandelstok.

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`



The code below shows us the result of passing this prompt to the model. The result is a langchain_core.messages.ai.AIMessage object. The 'content' parameter holds the AI message (the response) as a string. As requested by the format instructions in the prompt, this is a comma separated 'list' of values.
Passing this string to the CommaSeparatedListOutputParser results in an actual python list.

The model often returns more than the five requested keywords. Since this has minor consequences, I accept this behavior.

In [30]:
if verbose:
    response_keywords = model.invoke(P_keywords)
    parsed_response_keywords = csv_parser.parse(response_keywords.content)

    print(response_keywords)
    print(100 * '-')
    print(parsed_response_keywords)
    print(100 * '-')

content='Vasculaire dementie, Chronisch hartfalen, Begeleiding bij dagelijkse taken, Beperkte mobiliteit, Gebruik van wandelstok' response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 143, 'total_tokens': 181, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-47c1ad7f-bf10-44a0-b686-ba8a171e5df3-0' usage_metadata={'input_tokens': 143, 'output_tokens': 38, 'total_tokens': 181}
----------------------------------------------------------------------------------------------------
['Vasculaire dementie', 'Chronisch hartfalen', 'Begeleiding bij dagelijkse taken', 'Beperkte mobiliteit', 'Gebruik van wandelstok']
----------------------------------------------------------------------------------------------------


Chaining it all together

In [40]:
# Chain the prompt template with the model and the parser
chain_keywords = PT_keywords | model | csv_parser

if verbose:
    test_keywords = chain_keywords.invoke(
        {"profile": profile_as_string(test_client, display_name=False), 
         "scenario": scenario_as_string(test_client, period=1, sub_period=1)})
    print(test_keywords)

['Opname verpleeghuis', 'vasculaire dementie', 'hartfalen', 'beperkte mobiliteit', 'wandelstok.']


### Retrieving notes for the example library

The next step is to invoke the retriever with the generated keywords to obtain example notes. To add more ‘neutral’ notes, we also add the keywords ‘ADL’, ‘mobility’, and ‘food and drinks’ to the list.

Please note: The retriever utilizes an OpenAI embedding model (which is not free), which transforms the keywords into embeddings. These embeddings are then used to find similar notes in the initialized vector database, ensuring the retrieval of contextually relevant examples.

In [41]:
# Retrieves example notes from a retriever based on keywords
def get_example_notes(keywords, retriever):
    example_library = []
    example_library_topics = keywords
    # example_library_topics.extend(['adl', 'mobiliteit', 'eten en drinken'])
    for i in example_library_topics:
        docs = retriever.invoke(i)
        for d in docs:
            example_library.append(d.page_content.strip('"'))
    return example_library

if verbose:
    example_notes = get_example_notes(test_keywords, retriever)
    print(random.sample(example_notes,5))
    print(len(example_notes))

['Tijdens het wandelen is de rollator vastgelopen. Oude rollator, vervanging nodig.', 'Cli\\u00ebnt aangemoedigd om kleine wandelingen te maken om de mobiliteit te behouden. Stappenplan opgesteld om dit te stimuleren.', 'Fam wil graag een gesprek over de mogelijkheid van een hospice opname voor Dhr, team is bezig met overleg.', 'Familie van dhr. heeft gebeld met klachten over het voedsel in het verpleeghuis. Keukenteam op de hoogte gebracht voor verbetering.', 'Melding ontvangen van verpleging over een bloeduitstorting bij mevr, wondverpleegkundige ingeschakeld voor beoordeling en verzorging.']
100


In [42]:
# # aap = scenario_as_string(test_client, period=1, sub_period=1)
# aap = 'Opname in verpleeghuis vanwege vasculaire dementie en hartfalen'
# noot = retriever.invoke(aap)
# print(aap)
# print(sep_line)
# for doc in noot:
#     print(f"{doc.metadata['category']}: {doc.page_content}")

The example notes often contain gender-specific pronouns or titles. When these are included in the prompt the model tends to generate responses with incorrect pronouns for our client. Therefore, we need to filter the example library to exclude notes that use ‘mr’ or ‘mrs’.

In [43]:
# Determines the gender of a client based on their name
def determine_gender(name):
    if "Mevrouw" in name:
        return 'female'
    elif "Meneer" in name:
        return 'male'
    else:
        return 'unknown'
    
if verbose:
    print(f"The gender of {test_client['naam']} is: {determine_gender(test_client['naam'])}")

The gender of Meneer Emiel Bos is: male


In [44]:
# Filters example notes by the specified gender to ensure relevance
def filter_examples_by_gender(texts, gender_to_keep):
    if gender_to_keep == 'male':
        keywords = ['mw', 'mevr', 'mvr', 'mevrouw']
    elif gender_to_keep == 'female':
        keywords = ['dhr', 'meneer']
    else:
        return texts

    def contains_keywords(text):
        text_lower = text.lower()
        return any(keyword in text_lower for keyword in keywords)

    return [text for text in texts if not contains_keywords(text)]

if verbose:
    test_example_library = filter_examples_by_gender(texts=example_notes, gender_to_keep=determine_gender(test_client['naam']))
    [print('- '+item) for item in random.sample(test_example_library, 5)]
    print(f"\nOorspronkelijk aantal voorbeelden: {len(example_notes)}")
    print(f"Gefilterd aantal voorbeelden: {len(test_example_library)}")

- Familie van cli\u00ebnt heeft aangegeven dat ze een persoonlijk gesprek willen met de verpleegkundige over de dagelijkse gang van zaken.
- Verzoek van fam om foto's te sturen van activiteiten in het verpleeghuis. Foto's gemaakt en doorgestuurd via e-mail.
- - Fam ingelicht over positieve ontwikkelingen in gedrag en communicatie van dhr, goed nieuws delen.
- Dhr had vandaag duidelijk moeite met praten, leek de woorden niet goed te kunnen vinden
- Actieve deelname aan fysiotherapie voor mobiliteitsbehoud. Cli\u00ebnt toont grote veerkracht en motivatie tijdens de oefeningen.

Oorspronkelijk aantal voorbeelden: 100
Gefilterd aantal voorbeelden: 75


We'll be sampling a subset of this library

In [45]:
# Selects a random set of examples from the example library and returns as bulleted string
def sample_examples_as_string(example_library, num_items=3):
    random_items = random.sample(example_library, num_items)
    return '\n'.join(['- ' + item for item in random_items])

if verbose:
    sampled_examples_as_string = sample_examples_as_string(test_example_library, 5)
    print(sampled_examples_as_string)

- Familie informeerde naar dhr's sociale activiteiten in het verpleeghuis, overlegd met het activiteitenteam voor meer deelname.
- Opvallend dat cli\u00ebnt moeite heeft met het herkennen van bekende gezichten, mogelijk een teken van verergerende dementie.
- Tijdens het wandelen is de rollator vastgelopen. Oude rollator, vervanging nodig.
- Dhr had extra hulp nodig bij het aantrekken van zijn steunkousen, dit ging moeizaam maar uiteindelijk gelukt.
- Fam wil graag dat er regelmatig ge\u00ebvalueerd wordt over de zorg voor dhr, een evaluatiemoment plannen


Putting it all together in a function:

In [46]:
# Generates an example library for a client based on their profile and scenario
def create_example_library(row, scenario):
    profile_no_name = profile_as_string(row, display_name=False)
    client_gender = determine_gender(row['naam'])

    # Invoke the example library chain to get the keywords to search for example notes relevant to this client
    keywords = chain_keywords.invoke({"profile": profile_no_name, "scenario": scenario})
    example_notes = get_example_notes(keywords=keywords,retriever=retriever)
    example_library = filter_examples_by_gender(texts=example_notes, gender_to_keep=client_gender)

    return example_library

if verbose:
    example_library = create_example_library(test_client, scenario=scenario_as_string(test_client,period=1, sub_period=1))
    sampled_examples_as_string = sample_examples_as_string(example_library=example_library, num_items=3)
    print(sep_line)
    print(sampled_examples_as_string)

----------------------------------------------------------------------------------------------------
- Fam wil graag een gesprek over de mogelijkheid van dagbesteding buiten het verpleeghuis. Inplannen voor volgende week.
- Client had hulp nodig bij in- en uitstappen bed, fysieke ondersteuning geboden.
- Familie van dhr. heeft aangegeven dat ze ontevreden zijn over de communicatie in het verpleeghuis. Verbeterpunten bespreken in het team.


## Setting up the summary memory

### Memory prompt & chain

Adding memory is important because it allows the generated care notes to reflect ongoing developments in a client’s condition and care. This ensures that each new set of notes builds on previous information, maintaining continuity in the narrative of the client’s health and daily experiences.

I chose to implement memory by using a summary-based approach. This turned out to be more effective than passing the last notes directly, because it provides context through the profile and the summary, maintaining the storyline. Meanwhile, example notes are refreshed each time, preventing the model from becoming repetitive and producing the same structure repeatedly.

Initially, a summary is created that includes the client’s profile and the scenario of the first month. As new care notes are generated, this summary is updated to reflect the latest information. This updated summary is then used as the context for generating subsequent notes, ensuring that the model retains and incorporates past details. 

In the prompt, the model is asked to update the summary based on the previous summary and the newly generated client notes. 

[todo: a note about the temp, for now, I chose to keep it relatively high]

In [47]:
# # template for updating the client summary based on new care notes

# PT_memory = PromptTemplate(
#     template = """
# Hieronder staat:
# 1. Het profiel van een client(e) die verblijft op een psychogeriatrische afdeling van het verpleeghuis. 
# 2. Een samenvatting van het beloop tot de indexdatum
# 3. Nieuwe zorgrapportages vanaf de indexdatum

# PROFIEL:
# {profile}

# SAMENVATTING BELOOP TOT INDEXDATUM:
# {summary}

# NIEUWE RAPPORTAGES:
# {new_notes}

# Schrijf in één alinea een nieuwe samenvatting van het beloop. Neem belangrijke gebeurtenissen en zorgvraag uit de eerdere samenvatting over en vul aan met belangrijke gebeurtenissen en zorgvraag uit de rapportages. Neem de gegevens uit het profiel niet over in de samenvatting.

# In het antwoord dient uitsluitend de samenvatting van het beloop te staan, zonder aanvullende tekst.
# """,
#     input_variables=["profile", "summary", "new_notes"],
# )

# if verbose:
#     # Since we don't have any notes yet we'll be using example notes as new_notes
#     test_new_notes = sample_examples_as_string(example_library,9)
#     P_memory = PT_memory.format(profile=profile_as_string(test_client),
#                                 summary=scenario_as_string(test_client, month_no=1), 
#                                 new_notes=test_new_notes)
#     print(P_memory)

In [48]:
# chain_memory = PT_memory | model

# if verbose:
#     updated_summary = chain_memory.invoke({"profile": profile_as_string(test_client),
#                                            "summary": scenario_as_string(test_client, month_no=1), 
#                                            "new_notes": test_new_notes})
#     test_summary_m1 = updated_summary.content
#     pprint(test_summary_m1)

## Generating client notes

### Note generation prompts & chain

Our goal is populate the client record by generating care notes reflecting the client profile and describing the scenario. 
In practice, the number and length of notes per day vary based on clinical circumstances. Stable clients typically have fewer and shorter notes than ill or agitated clients. (I might add this functionality in the future.) Currently, I chose to generate three notes per day. Through trial and error, I found that the model can reliably generate nine notes per prompt. 
Having a scenario-twist every 3 days is not very realistic. I have scenario descriptions per 'month'. To populate a client record for an entire month, I could decide to break down the monthly scenario into smaller segments that fit the three-day prompt structure. I chose, however, to give the model the scenario in the first iteration and allowing it some creative freedom to build upon it, relying on the summaries to maintain continuity and build upon previous notes.

In [62]:
if verbose:
    # Let's study the scenario
    test_client_id = test_client['client_id']
    df_client_scenarios = df_scenarios.loc[df_scenarios['client_id'] == test_client_id, ['events_description', 'period', 'sub_period']]  
    counter = 1
    for i, r in df_client_scenarios.iterrows():
        print(f"p {r['period']} sp {r['sub_period']}: {r['events_description']}")
        # print(str(counter) + ' ' + r['events_description'])
        counter = counter + 1

p 1 sp 1: Meneer Bos wordt opgenomen in het verpleeghuis vanwege zijn vasculaire dementie en chronisch hartfalen. Hij krijgt begeleiding bij het wassen en aankleden, maar kan nog zelfstandig eten. Zijn mobiliteit is beperkt en hij maakt gebruik van een wandelstok.
p 1 sp 2: Meneer Bos begint langzaam te wennen aan zijn nieuwe omgeving in het verpleeghuis. Het zorgteam observeert zijn gedrag en past de zorg aan om hem comfortabel te laten voelen.
p 1 sp 3: Het zorgteam bouwt een vertrouwensband op met meneer Bos, waardoor hij zich meer op zijn gemak voelt. Er worden activiteiten georganiseerd die zijn interesse wekken en zijn cognitie stimuleren.
p 1 sp 4: Meneer Bos toont positieve reacties op de activiteiten en interacties met het zorgteam. Hij begint langzaam routine te vinden in zijn dagelijkse bezigheden en zijn stemming lijkt licht verbeterd.
p 2 sp 1: Gedurende deze periode vertoont meneer Bos tekenen van gewichtsverlies door zijn beperkte mobiliteit en lichamelijke klachten. Er 

In [67]:
sample_period = 4
sample_subperiod = 1

In [89]:
# template for generating care notes 
PT_get_notes = PromptTemplate(
    template="""Jouw taak als AI is om zorgrapportages te schrijven van een fictieve cliënt die verblijft op een psychogeriatrische afdeling van een verpleeghuis.

**INSTRUCTIES**:
- Schrijf rapportages voor zeven dagen, met drie rapportages per dag. In totaal zijn er 21 rapportages nodig. 
- Gebruik een informele, menselijke stijl met eenvoudige taal. Varieer de zinsopbouw en stijl. Vermijd technische termen en houd de rapportages natuurlijk en kort.
- Beschrijf de zorg zonder het profiel of de voorbeelden letterlijk te herhalen. 
- Vermijd het noemen van de naam, gebruik bijvoorbeeld 'cliënt', 'dhr' of 'mw'.
- De rapportages staan op zichzelf, individueel beschrijven ze één aspect (zoals ADL, mobiliteit, gedrag of somatiek). Samen moeten ze echter een verhaal vormen van het profiel en scenario.

**SCENARIO** voor de rapportages die je moet schrijven: 
{scenario}

**PROFIEL** van de cliënt:
{profile}

Voorbeeld rapportages:
- Cliënt is vanmorgen geholpen met wassen en aankleden.
{examples}

Nogmaals: Noem de naam van de client **niet**!

{format_instructions}
""",
    input_variables=["examples", "profile", "scenario"],
    partial_variables={"format_instructions": pyd_parser.get_format_instructions()},
)

if verbose:
    P_get_notes = PT_get_notes.format(
        examples = sample_examples_as_string(example_library, num_items=3), 
        profile = profile_as_string(test_client),
        scenario = scenario_as_string(test_client, sample_period, sample_subperiod)
        )
    print(P_get_notes)

Jouw taak als AI is om zorgrapportages te schrijven van een fictieve cliënt die verblijft op een psychogeriatrische afdeling van een verpleeghuis.

**INSTRUCTIES**:
- Schrijf rapportages voor zeven dagen, met drie rapportages per dag. In totaal zijn er 21 rapportages nodig. 
- Gebruik een informele, menselijke stijl met eenvoudige taal. Varieer de zinsopbouw en stijl. Vermijd technische termen en houd de rapportages natuurlijk en kort.
- Beschrijf de zorg zonder het profiel of de voorbeelden letterlijk te herhalen. 
- Vermijd het noemen van de naam, gebruik bijvoorbeeld 'cliënt', 'dhr' of 'mw'.
- De rapportages staan op zichzelf, individueel beschrijven ze één aspect (zoals ADL, mobiliteit, gedrag of somatiek). Samen moeten ze echter een verhaal vormen van het profiel en scenario.

**SCENARIO** voor de rapportages die je moet schrijven: 
Helaas ontwikkelt meneer Bos een pneumonie, waardoor zijn algehele gezondheid tijdelijk verslechtert. Er wordt met antibiotica behandeld en zijn condi

Now we create a chain. The output of the chain is a structured list of (nine) care notes for a client, encapsulated in an object called CareNotes. This object contains an attribute named notes, which is a list of individual CareNote entries. Each CareNote entry includes three pieces of information(dag, tijd, rapportage)

In [90]:
scenario_as_string(test_client, period=4, sub_period=1)

'Helaas ontwikkelt meneer Bos een pneumonie, waardoor zijn algehele gezondheid tijdelijk verslechtert. Er wordt met antibiotica behandeld en zijn conditie wordt nauwgezet in de gaten gehouden.'

In [93]:
# Create a chain of operations: prompt template -> model -> output parser
chain_get_notes = PT_get_notes | model | pyd_parser

if verbose:
    test_notes = chain_get_notes.invoke({"examples": sampled_examples_as_string, 
                                         "profile": profile_as_string(test_client),
                                         "scenario": scenario_as_string(test_client, period=sample_period, sub_period=sample_subperiod)})

    print('***The parsed result of model:')
    print(test_notes)

    print('\n***And these are the individual notes')
    for note in test_notes.notes:
        print(note)

***The parsed result of model:
notes=[CareNote(dag=1, tijd='08:00', rapportage='Cliënt had vanochtend hulp nodig bij het wassen en aankleden.'), CareNote(dag=1, tijd='12:30', rapportage='Mw. Bos heeft zelfstandig geluncht in de gemeenschappelijke ruimte.'), CareNote(dag=1, tijd='16:45', rapportage='Cliënt is met fysieke ondersteuning geholpen bij het verplaatsen naar de eetzaal voor het avondeten.'), CareNote(dag=2, tijd='09:15', rapportage='Cliënt leek vanochtend onrustig en gefrustreerd, extra aandacht geboden en kalmerende muziek afgespeeld.'), CareNote(dag=2, tijd='13:00', rapportage='Dhr. Bos heeft deelgenomen aan de handwerkactiviteit, waarbij hij geconcentreerd aan het werk was zonder interactie.'), CareNote(dag=2, tijd='17:30', rapportage='Familiebezoek in de middag, cliënt was zichtbaar opgewekt en betrokken tijdens het gesprek.'), CareNote(dag=3, tijd='10:00', rapportage='Mw. Bos maakte vanochtend een wandeling in de tuin onder begeleiding van een zorgverlener.'), CareNote(da

In [94]:
# Function formats the notes and returns them as a string for display
def notes_as_string(notes, simple_bulleted=True):
    note_strings = []
    for note in notes:
        if simple_bulleted:
            note_strings.append(f"- {note.rapportage}")
        else:
            note_strings.append(f"Dag {note.dag} ({note.tijd}): {note.rapportage}")
    return "\n".join(note_strings)

if verbose:
    print(notes_as_string(test_notes.notes, simple_bulleted=False))
    test_notes_m1 = notes_as_string(test_notes.notes)
    print(sep_line)
    print(test_notes_m1)

Dag 1 (08:00): Cliënt had vanochtend hulp nodig bij het wassen en aankleden.
Dag 1 (12:30): Mw. Bos heeft zelfstandig geluncht in de gemeenschappelijke ruimte.
Dag 1 (16:45): Cliënt is met fysieke ondersteuning geholpen bij het verplaatsen naar de eetzaal voor het avondeten.
Dag 2 (09:15): Cliënt leek vanochtend onrustig en gefrustreerd, extra aandacht geboden en kalmerende muziek afgespeeld.
Dag 2 (13:00): Dhr. Bos heeft deelgenomen aan de handwerkactiviteit, waarbij hij geconcentreerd aan het werk was zonder interactie.
Dag 2 (17:30): Familiebezoek in de middag, cliënt was zichtbaar opgewekt en betrokken tijdens het gesprek.
Dag 3 (10:00): Mw. Bos maakte vanochtend een wandeling in de tuin onder begeleiding van een zorgverlener.
Dag 3 (14:45): Cliënt was vandaag onrustig en verbaal agressief richting medebewoners, afleiding gezocht met een groepsactiviteit.
Dag 3 (18:00): Dhr. Bos is bij het avondeten gestimuleerd om zelfstandig te eten met kleine, behapbare porties.
Dag 4 (07:30): V

In [80]:
# # Seeing some iterations explicitly written out can make the flow of the process clearer, especially when tracking the sequence of actions and understanding the logic at each step. 
# if verbose:
#     print('PROFILE')
#     print(profile_as_string(test_client))
#     print('\nSCENARIO: '+ scenario_as_string(test_client,month_no=1))
#     print('\nNOTES')
#     print(test_notes_m1)

#     # Now have the model create a new summary 
#     updated_summary = chain_memory.invoke({"profile": profile_as_string(test_client),
#                                            # Initially, there is no summary
#                                            "summary": scenario_as_string(test_client, month_no=1), 
#                                            "new_notes": test_notes_m1})
#     test_summary_m1 = updated_summary.content
#     print('\nSUMMARY')
#     pprint(test_summary_m1)

#     # And again some notes
#     test_notes = chain_get_notes.invoke({"examples": sample_examples_as_string(example_library, 3), 
#                                          "profile": profile_as_string(test_client),
#                                          "summary": test_summary_m1, 
#                                          "scenario": "Bouw voort op de gegevens uit het Profiel en het Beloop"})

#     test_notes_m2 = notes_as_string(test_notes.notes)
#     print('\nSCENARIO: '+ "Bouw voort op de gegevens uit het Profiel en het Beloop")
#     print('\nNOTES')
#     print(test_notes_m2)

#     # a new summary 
#     updated_summary = chain_memory.invoke({"profile": profile_as_string(test_client),
#                                            "summary": test_summary_m1, 
#                                            "new_notes": test_notes_m2})
#     test_summary_m2 = updated_summary.content
#     print('\nSUMMARY')
#     pprint(test_summary_m2)

#     # And again some notes
#     test_notes = chain_get_notes.invoke({"examples": sample_examples_as_string(example_library, 3), 
#                                          "profile": profile_as_string(test_client),
#                                          "summary": test_summary_m2, 
#                                          "scenario": scenario_as_string(test_client, month_no=2)})

#     test_notes_m3 = notes_as_string(test_notes.notes)
#     print('\nSCENARIO: '+ scenario_as_string(test_client,month_no=2))
#     print('\nNOTES')
#     print(test_notes_m3)


### Fuctions to populate the client records

In [ ]:
# Generate care notes for a client over a specified number of iterations and update the client summary
def generate_care_notes(summary_list, notes_list, profile_row, month_no, example_library, num_iterations=5, num_examples=3):  
    """
    Returns:
    - Updated list of summaries.
    - Updated list of care notes.
    """
    profile = profile_as_string(profile_row)
    scenario = scenario_as_string(profile_row, month_no=month_no)
    client_id = profile_row['client_id']
    summary = summary_list[-1]

    for i in range(num_iterations):
        iteration = i + 1
        try:
            print(f'Iteration {iteration}')
            if iteration > 1:
                # Update the scenario to let the model build upon the scenario
                scenario = "Bouw voort op de gegevens uit het Profiel en het Beloop."

            # Sample examples from the example library
            examples = sample_examples_as_string(example_library, num_examples)

            # Generate care notes using the model and the example library
            # There are frequent 'Invalid json output' errors. In that case, try again
            try:
                result_notes = chain_get_notes.invoke({
                    "examples": examples,
                    "profile": profile,
                    "summary": summary,
                    "scenario": scenario
                })
            except Exception as e:
                # Try once more in case of failure
                print(f"Error in iteration {iteration}, retrying: {e}")
                result_notes = chain_get_notes.invoke({
                    "examples": examples,
                    "profile": profile,
                    "summary": summary,
                    "scenario": scenario
                })
                print("Retry successful")

            # Update the summary based on the new care notes
            result_memory = chain_memory.invoke({
                 "profile": profile,
                 "summary": summary,
                 "new_notes": notes_as_string(result_notes.notes)
            })

            # Add generated notes to the notes list
            for note in result_notes.notes:
                notes_list.append({
                    "client_id": client_id,
                    "month": month_no,
                    "iteration": iteration,
                    "dag": note.dag,
                    "tijd": note.tijd,
                    "rapportage": note.rapportage,
                })

            # add_notes_to_list(result_notes.notes, notes_list, client_id, month_no, iteration)

            # Update the memory with new notes and generate a new summary
            summary = result_memory.content

            # Add the updated summary to the summary list 
            summary_list.append({
                "client_id": client_id,
                "month": month_no,
                "iteration": iteration,
                "summary": summary,
            })

        except Exception as e:
            print(f"Error in iteration {iteration}: {e}")
            continue

    return summary_list, notes_list

if verbose:
    notes_list = []
    summary_list = []
    summary_list.append({
        "client_id": test_client['client_id'],
        "month": 0,
        "iteration": 0,
        "summary": scenario_as_string(test_client, month_no=1),
        })

    summary_list, notes_list = generate_care_notes(
        summary_list=summary_list, 
        notes_list=notes_list, 
        profile_row=test_client, 
        example_library=example_library,
        month_no=1,
        num_iterations=1,
        num_examples=3)
    
    for cs in summary_list:
        print(cs)
    print(100*'-')
    for n in notes_list:
        print(n)
    

Next, let's have a look at the generation of care notes and summaries for a single client. We need to process each client’s data individually. This involves iterating through their associated scenarios and generating relevant notes. 

In [ ]:
# Processe a single client to generate care notes and summaries.
def process_client(profile_row, df_scenarios, num_iterations=5, num_examples=3):
    all_notes_list = []
    all_summaries_list = []

    try:
        with get_openai_callback() as cb:
            print(f"Processing client: {profile_row['naam']}")

            summary_list = []
            notes_list = []

            client_id = profile_row['client_id']
            # As initial summary, we take the scenario of the first month
            summary_list.append({
                "client_id": profile_row['client_id'],
                "month": 0,
                "iteration": 0,
                "summary": scenario_as_string(profile_row=profile_row, month_no=1),
                })

            # Select the scenario rows for the client
            df_client_scenarios = df_scenarios.loc[df_scenarios['client_id'] == client_id, ['journey', 'month']]  

            num_months = len(df_client_scenarios)

            month_no = 1    
            for i, month_scenario in df_client_scenarios.iterrows():  
                scenario = scenario_as_string(profile_row, month_no)
                example_library = create_example_library(profile_row, scenario)

                print(f'Generating notes for month: {month_no} of {num_months} for client {client_id}')
                summary_list, notes_list = generate_care_notes(
                    summary_list=summary_list,
                    notes_list=notes_list, 
                    profile_row=profile_row,
                    month_no=month_no,
                    example_library=example_library,
                    num_iterations=num_iterations,
                    num_examples=num_examples,
                    )
                month_no += 1

            # Add client_id to notes and summaries
            for note in notes_list:
                note['client_id'] = client_id
            for summary in summary_list:
                all_summaries_list.append({'client_id': client_id, 'summary': summary, 'month': month_no})  

            all_notes_list.extend(notes_list)
            print(cb)

    except Exception as e:
        print(f"Error processing client {profile_row['naam']}: {e}")

    return all_notes_list, all_summaries_list

if verbose:
    notes_list, summaries_list = process_client(profile_row=test_client, df_scenarios=df_scenarios, num_iterations=2, num_examples=3)
    for cs in summaries_list:
        print(cs)

    for n in notes_list:
        print(n)

In [ ]:
# Iterate through all clients, processes each one, and saves the generated notes and summaries to CSV files.
def process_clients(df_profiles, df_scenarios):
    all_notes_list = []
    all_summaries_list = []

    for idx, row in df_profiles.iterrows():
        try:
            notes, summaries = process_client(row, df_scenarios)
            all_notes_list.extend(notes)
            all_summaries_list.extend(summaries)

            df_notes = pd.DataFrame(all_notes_list)
            df_summaries = pd.DataFrame(all_summaries_list)

            # save after each client to prevent having to start over in case of an error
            df_notes.to_csv(path_notes, index=False)
            df_summaries.to_csv(path_summaries, index=False)
        except Exception as e:
            print(f"Error processing client: {e}")

process_clients(df_profiles=df_profiles, df_scenarios=df_scenarios)


In [ ]:
def update_dag_counter(df):
    """
    This function updates the 'dag' column in df such that it maintains a running counter per client and per month.
    The counter starts at 1 and increments by 1 each time the 'dag' value changes. The counter resets to 1 for each new client and month.
    """
    # Add a column to shift 'dag' values by one row within each group of client_id and month
    df['dag_shift'] = df.groupby(['client_id', 'month'])['dag'].shift(1)
    # Create a column indicating if 'dag' has changed compared to the previous row
    df['dag_changed'] = (df['dag'] != df['dag_shift']).astype(int)
    # Create a column indicating the start of a new group (client_id and month)
    df['group_changed'] = df.groupby(['client_id', 'month']).cumcount() == 0
    
    # Update 'group_changed' to be False if 'dag_shift' is NaN
    df['group_changed'] = df['group_changed'] & df['dag_shift'].notna()
    # Create the counter ('teller') by cumulatively summing 'dag_changed' within each group and adding 'group_changed'
    df['dag'] = df.groupby(['client_id', 'month'])['dag_changed'].cumsum() + df['group_changed']
    
    # Remove the temporary columns used for calculations
    df.drop(columns=['dag_shift', 'dag_changed', 'group_changed'], inplace=True)
    
    return df


In [ ]:
dfn = pd.read_csv(path_notes)
dfn = update_dag_counter(dfn)
dfn.to_csv(path_notes, index=False)